In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

# HDRUK scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup


In [2]:
import requests
from bs4 import BeautifulSoup
from pyconceptlibraryclient import Client
import re
 
# Base URL
base_url = "https://phenotypes.healthdatagateway.org/phenotypes/"

#Client
client = Client(public=True)

# Loop through all pages


def hdruk_scrapping(base_url,client):
    hdruk = []
    response = requests.get(base_url)
    if response.status_code == 200: 
        soup = BeautifulSoup(response.content, 'html.parser')
        page_items = soup.select("ul.pagination-container__pages li")


        page_numbers = []
        for item in page_items:
            if "divider" in item.get("class", []):
                continue
            text = item.get_text(strip=True)
            if text.isdigit():
                page_numbers.append(int(text))

        page_number = min(page_numbers)

        for page in range(1, page_number + 1):
            all_titles = []
            res = requests.get(f"{base_url}?page={page}")
            soup = BeautifulSoup(res.content, "html.parser")
            
            links = soup.find_all('a', class_="entity-card__click",href = True)
            all_titles = [link.find('h3',class_ = "entity-card__title").string for link in links]

            for title in all_titles:
                phenotype_text = title.text.strip()
                phenotype_id = phenotype_text.split()[0] 
                phenotype_detail = client.phenotypes.get_detail(phenotype_id)
                print(phenotype_id)

                try:
                    phenotype_concept = client.phenotypes.get_codelist(phenotype_id)
                except:
                    phenotype_concept = []

                hdruk_dictionary = {}
                hdruk_dictionary['phenotype_id']= phenotype_detail[0]['phenotype_id']
                hdruk_dictionary['version_id'] = phenotype_detail[0]['phenotype_version_id']
                hdruk_dictionary['details'] = phenotype_detail
                hdruk_dictionary['codelist'] = phenotype_concept
                    
                hdruk.append(hdruk_dictionary)

        return hdruk
    else:
        return f"Failed to access HDRUK page"
            



In [4]:
hdruk = hdruk_scrapping(base_url,client)

PH1
PH2
PH3
PH4
PH5
PH6
PH7
PH8
PH9
PH10
PH11
PH12
PH13
PH14
PH15
PH16
PH17
PH18
PH19
PH20
PH21
PH22
PH23
PH24
PH25
PH26
PH27
PH28
PH29
PH30
PH31
PH32
PH33
PH34
PH35
PH36
PH37
PH38
PH39
PH40
PH41
PH42
PH43
PH44
PH45
PH46
PH47
PH48
PH49
PH50
PH51
PH52
PH53
PH54
PH55
PH56
PH57
PH58
PH59
PH60
PH61
PH62
PH63
PH64
PH65
PH66
PH67
PH68
PH69
PH70
PH71
PH72
PH73
PH74
PH75
PH76
PH77
PH78
PH79
PH80
PH81
PH82
PH83
PH84
PH85
PH86
PH87
PH88
PH89
PH90
PH91
PH92
PH93
PH94
PH95
PH96
PH97
PH98
PH99
PH100
PH101
PH102
PH103
PH104
PH105
PH106
PH107
PH108
PH109
PH110
PH111
PH112
PH113
PH114
PH115
PH116
PH117
PH118
PH119
PH120
PH121
PH122
PH123
PH124
PH125
PH126
PH127
PH128
PH129
PH130
PH131
PH132
PH133
PH134
PH135
PH136
PH137
PH138
PH139
PH140
PH141
PH142
PH143
PH144
PH145
PH146
PH147
PH148
PH149
PH150
PH151
PH152
PH153
PH154
PH155
PH156
PH157
PH158
PH159
PH160
PH161
PH162
PH163
PH164
PH165
PH166
PH167
PH168
PH169
PH170
PH171
PH172
PH173
PH174
PH175
PH176
PH177
PH178
PH179
PH180
PH181
PH182
PH183
PH184
PH18

ConnectTimeout: HTTPSConnectionPool(host='conceptlibrary.saildatabank.com', port=443): Max retries exceeded with url: /api/v1/phenotypes/PH958/detail (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001877E205D90>, 'Connection to conceptlibrary.saildatabank.com timed out. (connect timeout=None)'))

In [ ]:
from src.utils import save_json
from src.config import RAW_DIR,HDRUK_URL
try:

    save_json(RAW_DIR,hdruk,'HDRUK')
    print("HDRUK saved successfully")
except:
    print("HDRUK not saved")

In [ ]:
from src.utils import get_hdruk_client
from src.utils import save_json
from src.config import RAW_DIR,HDRUK_URL

In [ ]:

def main():
    base_url = HDRUK_URL
    client = get_hdruk_client()
    hdruk = hdruk_scrapping(base_url,client)
    # save_json(RAW_DIR,hdruk,'HDRUK')
    return hdruk
hdruk = main()


In [ ]:
import os
import json
from datetime import datetime
import re


def date_extract(date):
    if date == "" or date is None:
        event_start_date = "NA"
        event_end_date = "NA"
    else:
        date1 = date.replace(' - ','-').replace(' to ','-').replace(' and ','-')
        date2 = date1.replace('Start', "NA").replace('start', "NA")
        
        if not re.match(r'\d{4}-\d{4}',date2):
            Start_date = date2.split("-")[0]
            End_date = date2.split("-")[-1]

            if Start_date != 'NA':
                try:
                    start_date = datetime.strptime(Start_date,"%Y/%m/%d") if re.match(r'^\d{4}',Start_date) else datetime.strptime(Start_date,"%d/%m/%Y")
                    event_start_date = datetime.strftime(start_date,"%Y-%m-%dT%H:%M:%S.00Z")
                except:
                    event_start_date = 'Invalid date'
            else:
                event_start_date = 'NA'

            if End_date != 'NA':
                try:
                    end_date = datetime.strptime(End_date,"%Y/%m/%d") if re.match(r'^\d{4}',End_date) else datetime.strptime(End_date,"%d/%m/%Y")
                    event_end_date = datetime.strftime(end_date,"%Y-%m-%dT%H:%M:%S.00Z")
                except:
                    event_end_date = 'Invalid date'  
            else:
                event_end_date = 'NA'

        else:

            Start_date = date2.split("-")[0]
            End_date = date2.split("-")[-1]

            start_date = datetime.strptime(Start_date,"%Y") 
            end_date = datetime.strptime(End_date,"%Y")     
            event_start_date = datetime.strftime(start_date,"%Y-%m-%dT%H:%M:%S.00Z")
            event_end_date = datetime.strftime(end_date,"%Y-%m-%dT%H:%M:%S.00Z")
        
    return event_start_date,event_end_date

def get_detail(hdruk):
    hdruk_detail = []

    for data in hdruk:
        detail_dictionary = {}
        detail = data['details'][0]
        detail_dictionary["Phenotype_id"] = detail['phenotype_id'] if detail['phenotype_id'] else 'NA'
        # print(d["Phenotype_id"])
        detail_dictionary["Phenotype_version_id"] = int(detail['phenotype_version_id']) if detail['phenotype_version_id'] else 'NA'
        detail_dictionary["Name"] = detail['name'] if detail['name'] else 'NA'
        detail_dictionary["Defination"] = detail['definition'] if detail['definition'] else 'NA'
        detail_dictionary["Implementation"] = detail['implementation'] if detail['implementation'] else 'NA'
        detail_dictionary["Publications"] = detail['publications'] if detail['publications'] else 'NA'
        detail_dictionary["Validation"] = detail["validation"] if detail["validation"] else 'NA'
        detail_dictionary["Citation_requirements"] = detail["citation_requirements"] if detail["citation_requirements"] else 'NA'
        detail_dictionary["Created"] = detail["created"] if detail["created"] else 'NA'
        detail_dictionary["author"] = detail["author"] if detail["author"] else 'NA'
        detail_dictionary["Collections"] = [name ['name'] for name in detail['collections']] if detail['collections'] else 'NA'
        detail_dictionary["Tags"] = detail["tags"] if detail["tags"] else 'NA'
        detail_dictionary["Group"] = detail["group"] if detail["group"] else 'NA'
        detail_dictionary["Group_access"] = int(detail["group_access"]) if detail["group_access"] else 'NA'
        detail_dictionary["World_access"] = int(detail["world_access"]) if detail["world_access"] else 'NA'
        detail_dictionary["Updated"] = detail["updated"] if detail["updated"] else 'NA'
        detail_dictionary["Sex"] = detail['sex'][0]["name"] if detail['sex'] else 'NA'
        detail_dictionary["Type"] = detail['type'][0]["name"] if detail['type'] else 'NA'
        detail_dictionary["Phenoflow_id"] = int(detail['phenoflowid']) if detail['phenoflowid'] else 'NA'
        try:
            detail_dictionary["Data_sources"] = {item['name'] : item['url'] for item in detail['data_sources']} if 'url' in detail['data_sources'][0].keys() else 'NA'
        except: 
            detail_dictionary['Data_sources'] = 'NA'
        detail_dictionary["Coding_system"] = detail['coding_system'][0]["name"] if detail['coding_system'] else 'NA'
        detail_dictionary['Event_start_date'],detail_dictionary['Event_end_date'] = date_extract(detail['event_date_range']) 
        detail_dictionary["Status"] = int(detail["status"]) if detail["status"] else 'NA'
        detail_dictionary["Is_deleted"] = detail["is_deleted"] if detail["is_deleted"] else 'NA'
        detail_dictionary["Owner"] = detail["owner"] if detail["owner"] else 'NA'
        
        hdruk_detail.append(detail_dictionary)   

    sorted_detail = sorted(hdruk_detail,key=lambda x:x['Phenotype_id'])
    i = 0
    for detail in sorted_detail:
        i+=1
        detail['PID'] = f'HP{i:06d}'

    return sorted_detail


def get_concept(hdruk,detail):
    hdruk_concept = []
    id_list = []
    codelist = [data['codelist'] for data in hdruk]

    for data in codelist:
        for dic in data:
            if dic["id"] not in id_list:
                concept_dictionary = {}
                id_list.append(dic["id"])
                concept_dictionary["Description"] = dic['description'].strip().replace("\\", "")
                concept_dictionary["ID"] = dic["id"]
                concept_dictionary["Concept_id"] = [dic['concept_id']]
                concept_dictionary["Concept_history_id"] = [dic['concept_history_id']]
                concept_dictionary["Concept_history_date"] = [dic['concept_history_date']]
                concept_dictionary["Component_id"] = [dic["component_id"]]
                concept_dictionary["Component_history_id"] = [dic["component_history_id"]]
                concept_dictionary["Logical_type"] = [dic["logical_type"]]
                concept_dictionary["Codelist_id"] = [dic["codelist_id"]]
                concept_dictionary["Codelist_history_id"] = [dic["codelist_history_id"]]
                concept_dictionary["Code"] = [dic["code"]]
                concept_dictionary["Coding_system_id"] = [dic["coding_system"]["id"]]
                concept_dictionary["Coding_system_name"] = [dic["coding_system"]["name"]]
                concept_dictionary["Coding_system_description"] =  [dic["coding_system"]["description"]]
                concept_dictionary["Phenotype_id"] = [dic["phenotype_id"]]
                concept_dictionary["Phenotype_version_id"] = [dic["phenotype_version_id"]]
                concept_dictionary["Phenotype_name"] = [dic["phenotype_name"]]
                concept_dictionary["PIDs"] = [P["PID"] for P in detail if P['Phenotype_id'] == dic["phenotype_id"]]
                hdruk_concept.append(concept_dictionary)
            else:
                for concept in hdruk_concept:
                    if concept['ID'] == dic["id"]:
                        concept['Concept_id'].append(dic['concept_id']) 
                        concept['Concept_history_id'].append(dic['concept_history_id'])
                        concept['Concept_history_date'].append(dic['concept_history_date'])
                        concept['Component_id'].append(dic['component_id'])
                        concept['Component_history_id'].append(dic['component_history_id'])
                        concept['Logical_type'].append(dic['logical_type'])
                        concept['Codelist_id'].append(dic['codelist_id'])
                        concept['Codelist_history_id'].append(dic['codelist_history_id'])
                        concept['Code'].append(dic['code'])
                        concept['Coding_system_id'].append(dic['coding_system']['id'])
                        concept['Coding_system_name'].append(dic['coding_system']['name'])
                        concept['Coding_system_description'].append(dic['coding_system']['description'])
                        concept['Phenotype_id'].append(dic['phenotype_id'])
                        concept['Phenotype_version_id'].append(dic['phenotype_version_id'])
                        concept['Phenotype_name'].append(dic['phenotype_name'])
                        concept['PIDs'].extend([P["PID"] for P in detail if P['Phenotype_id'] == dic["phenotype_id"]])

    concept_sorted = sorted(hdruk_concept,key = lambda x: x['Description'])
    i = 0
    for concept in concept_sorted:
        i += 1
        concept['CID'] = f'HC{i:06d}'

    return concept_sorted


In [ ]:
from src.config import HDRUK_DIR
from src.utils import save_detail,save_concept

In [ ]:


def main():
    # with open(rf'{dir}\hdruk_phenotypes.json', 'r') as file:
    #     file = file.read()
    #     hdruk = json.loads(file)
    detail = get_detail(hdruk)
    concept = get_concept(hdruk,detail)
    save_detail(HDRUK_DIR,detail,'HDRUK')
    save_concept(HDRUK_DIR,concept,'HDRUK')
    return detail,concept

hdruk_detail,hdruk_concept = main()

In [ ]:
import requests
from bs4 import BeautifulSoup
from pyconceptlibraryclient import client
base_url="https://phenotypes.healthdatagateway.org/phenotypes?page="

def hdruk_scrapping(base_url, client):
    hdruk = []
    page = 1

    while True:
        url = f"{base_url}{page}"
        response = requests.get(url)
        print("Page:", page)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract phenotype titles
            titles = soup.find_all("h3", class_="entity-card__title")

            for title in titles:
                phenotype_text = title.text.strip()
                phenotype_id = phenotype_text.split()[0] 

                phenotype_detail = client.phenotypes.get_detail(phenotype_id)
                try:
                    phenotype_concept = client.phenotypes.get_codelist(phenotype_id)
                except:
                    phenotype_concept = []

                phenotype_dictionary = {
                    'phenotype_id': phenotype_detail[0]['phenotype_id'],
                    'version_id': phenotype_detail[0]['phenotype_version_id'],
                    'details': phenotype_detail,
                    'codelist': phenotype_concept
                }
                hdruk.append(phenotype_dictionary)    

            # Check if "next" button is disabled
            next_button = soup.select_one("ul.pagination-container__next li.disabled")
            if next_button:
                print("Reached the last page.")
                break
            else:
                page += 1
        else:
            print(f"Failed to access page {page}")
            break

    return hdruk


SyntaxError: invalid syntax (1235623651.py, line 4)

In [ ]:
hdruk_scrapping(base_url, client)